<a href="https://colab.research.google.com/github/mostafa-ja/sample/blob/master/VGG_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn

In [12]:
vgg_cfgs = {
    "vgg11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "vgg13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "vgg16": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
    "vgg19": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"],
    }

In [5]:
def make_layers(architecture, in_channels):
  in_channels = in_channels
  layers = nn.Sequential()
  for x in architecture:
    if x=='M':
      layers.append(nn.MaxPool2d(2,stride=2))
    elif type(x)==int :
      out_channels = x
      layers.append(nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=1,padding=1))
      layers.append(nn.BatchNorm2d(num_features=out_channels))
      layers.append(nn.ReLU())
      in_channels = x

  return layers


**Second way to define def make_layers()**



```
def make_layers(architecture, in_channels):
  in_channels = in_channels
  layers = []
  for x in architecture:
    if x=='M':
      layers += [nn.MaxPool2d(2,stride=2)]
    elif type(x)==int:
      out_channels = x
      layers += [nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=1,padding=1),
                 nn.BatchNorm2d(num_features=out_channels),
                 nn.ReLU()]
      in_channels = x

  return nn.Sequential(*layers)

```




In [14]:
class VGGNet(nn.Module):
  def __init__(self,architecture, in_channels,num_classes):
    super().__init__()
    self.conv_layers = make_layers(architecture,in_channels)
    self.fcs = nn.Sequential(nn.Linear(512*7*7, 4096),
                             nn.ReLU(),
                             nn.Dropout(0.5),
                             nn.Linear(4096, 4096),
                             nn.ReLU(),
                             nn.Dropout(0.5),
                             nn.Linear(4096,num_classes),
                             )
    
  def forward(self,x):
    out = self.conv_layers(x)
    out = out.reshape(out.shape[0],-1)   # OR out = torch.flatten(out , start_dim=1)
    out = self.fcs(out)
    return out


In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = VGGNet(vgg_cfgs["vgg16"],3,1000).to(device)
print(model)

VGGNet(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), paddi

In [16]:
x = torch.randn(4,3,224,224).to(device)  # Mini batch size = 4

print(model(x).shape)

torch.Size([4, 1000])
